In [2]:
from transformers import AutoTokenizer, OPTForCausalLM

/Users/vasu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
pip install transformers accelerate

In [ ]:
pip install --upgrade transformers


In [3]:
tokenizer = AutoTokenizer.from_pretrained("facebook/galactica-1.3b")
model = OPTForCausalLM.from_pretrained("facebook/galactica-1.3b", device_map="auto",offload_folder="/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/language_models/offload/galactica-1.3b")


In [5]:
# Set device to CPU
device = "cpu"

# Generate text on CPU
input_text = "Breast Cancer is"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# Control the maximum number of tokens in the generated text

outputs = model.generate(input_ids.to(device), max_length=100)


# Decode generated output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Breast Cancer is the most common cancer in women worldwide. The incidence of breast cancer is increasing in developing countries. The aim of this study was to evaluate the effect of the combination of tamoxifen and raloxifene on the expression of estrogen receptor (ER), progesterone receptor (PR), and human epidermal growth factor receptor 2 (HER2) in breast cancer.

